### Speed Humps Analysis (2011-2017)

#### Database: DOTDEVRHPGSQL01
#### Tables: speed_humps

#### 1. Year with most speed humps
#### 2. All mfts with either sip corridor or intersection projects of study year range
#### 3. Writing all sip project mfts of study year range from sip database to crash database
#### 4. Sipless Single Year MFT Speed Humps of study year
#### 5. Control Group: All mfts with no sips, no Speed humps and outside of study year range 
#### 6. All Injuries by node from FORMS and TAMS Data
#### 7. Before-Afters for control and study group




In [49]:
from ris import pysqldb
from IPython.display import clear_output
from collections import defaultdict
from collections import defaultdict, namedtuple
import datetime 
import pandas as pd
import numpy as np
import os
from sqlalchemy import create_engine

clear_output()
timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')
ts = datetime.datetime.now().strftime('%Y-%m-%d')

print 'Notebook run: {}'.format(timestamp)
print os.getcwd()

Notebook run: 2020-03-02 16:42
E:\RIS\Staff Folders\Samuel\Requests\AG\Speed Humps


In [3]:
#Database connection
cdb = pysqldb.DbConnect(server='DOTDEVRHPGSQL01', database='CRASHDATA', type='PG')
sdb = pysqldb.DbConnect(server='dotpgsql01', database='sip', type='PG')
sqldb = pysqldb.DbConnect(server='dot55sql01', database='forms', type='MS', user = 'arcgis', password = 'arcgis')


Additional database connection details required:
User name (crashdata):soge
Password (crashdata)········
Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection established 2020-03-02 15:05:04

Additional database connection details required:
User name (sip):soge
Password (sip)········
Database connection (PG) to sip on dotpgsql01 - user: soge 
Connection established 2020-03-02 15:05:08
	Missing SQL Server Native Client 10.0                 datetime2 will not be interpreted correctly

Database connection (MS) to forms on dot55sql01 - user: arcgis 
Connection established 2020-03-02 15:05:09
Query run in 25000 microseconds
	Missing SQL Server Native Client 10.0                 datetime2 will not be interpreted correctly



In [4]:
my_study_year = 2014
study_year_min = my_study_year-3
study_year_max = my_study_year+3

### 1. Year with most speed humps

In [5]:
result = cdb.dfquery("""    

SELECT left(date_installed::text,4) as year, count(distinct lion.mft)
FROM public.speed_humps sh
join archive."18d.2019-11-13_lion" lion
on sh.segmentid = lion.segmentid 
where left(date_installed::text,4)::int between 2008 and 2014
group by year
order by year

""")

In [6]:
result

,year,count
0,2008,129
1,2009,85
2,2010,153
3,2011,106
4,2012,221
5,2013,265
6,2014,361


### 2. All mfts with either sip corridor or intersection projects of study year range

In [7]:
sip_projs = sdb.dfquery("""

--All corr/itx mfts with completed sips btwn {study_year_min}-{study_year_max}

select * from (

--All geometries that have mfts with completed sips between {study_year_min}-{study_year_max}
SELECT distinct coalesce(lion.mft,0)::text mft, coalesce(masteridfr,0)::text masteridfrom, coalesce(masteridto,0)::text masteridto, lion.geom
FROM public.sip_projects sp
join public.sip_projects_geo spg
on sp.pid=spg.pid_fk
join clion lion
on spg.segmentid::int = lion.segmentid::int
where sp.status= '15'
and spg.nodeid=0
and left(sip_year::text,4)::int between {study_year_min} and {study_year_max}
and left(end_date::text,4)::int between {study_year_min} and {study_year_max}

) sip_corr_mfts


union 

select * from (


--Completed Sip masterids to mft/geometries between {study_year_min}-{study_year_max}
with data as(


SELECT sp.pid, sp.pjct_name, sp.pm, sp.sip_year, sp.start_date, sp.end_date, spg.nodeid, st_setsrid(spg.geom,2263) geom
FROM public.sip_projects sp
join public.sip_projects_geo spg
on sp.pid=spg.pid_fk
where sp.status= '15'
and spg.segmentid=0
and left(sp.sip_year::text,4)::int between {study_year_min} and {study_year_max}
and left(sp.end_date::text,4)::int between {study_year_min} and {study_year_max}
)


SELECT distinct coalesce(clfr.mft,0)::text mft, coalesce(masteridfr,0)::text masteridfrom, coalesce(masteridto,0)::text masteridto, clfr.geom from data
left join clion_node cln
on data.nodeid::int = cln.nodeid::int
left join  clion clfr
on cln.masterid = clfr.masteridfr

union 

SELECT distinct coalesce(clto.mft,0)::text mft, coalesce(masteridfr,0)::text masteridfrom, coalesce(masteridto,0)::text masteridto, clto.geom from data
left join clion_node cln
on data.nodeid::int = cln.nodeid::int
left join clion clto
on cln.masterid = clto.masteridto


) sip_itx_mfts

""".format(study_year_min=study_year_min,study_year_max=study_year_max) 
                       )

### 3. Writing all sip project mfts of study year range from sip database to crash database

#### Runtime 5 mins

In [8]:
cdb.connect()
cdb.dataframe_to_table_schema(sip_projs, 'sip_projs', schema='working', overwrite = True)

Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection established 2020-03-02 15:06:27


[['mft', 'varchar (500)'],
 ['masteridfrom', 'varchar (500)'],
 ['masteridto', 'varchar (500)'],
 ['geom', 'varchar (500)']]

In [11]:
cdb.connect()
#input_schema = cdb.dataframe_to_table_schema(sip_projs, 'sip_projs', schema='working', overwrite = True)
sh_sips = cdb.dataframe_to_table(sip_projs, 'sip_projs', schema= 'working', overwrite = True)


Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection established 2020-03-02 15:12:18
Reading data into Database



85it [00:01, 43.26it/s]

Failure:

- Query run 2020-03-02 15:12:20.686000
	
                INSERT INTO working.sip_projs ("mft", "masteridfrom", "masteridto", "geom")
                VALUES ('0', '0', '0', '0105000020D70800000100000001020000000F0000000004B7EB6F462E410032EEBE099D05410050BC4E67462E41001670A1969C0541B9CCB33F4B462E41E2B2E0C84E9B05410F96587D2A462E415C6F89FD0D9A0541802CD42305462E41004C3153D5980541FEFE334CEA452E41AD7BBE0B029805414B85A32FD2452E4163E6F88F29970541F95339DEBC452E4197D252704C96054180D22E66AA452E41002057406B950541BCA7A2069A452E41BAB81DB1949405418ADD19548C452E4157D92527BB93054100492E5781452E410080082BDF920541175359D57C452E41553D15931192054156AEA3087C452E41866A81374391054180AA1CF27E452E410000CC2975900541')
            
Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection closed 2020-03-02 15:12:20
Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection established 2020-03-02 15:12:20


406it [00:09, 43.89it/s]

Failure:

- Query run 2020-03-02 15:12:27.809000
	
                INSERT INTO working.sip_projs ("mft", "masteridfrom", "masteridto", "geom")
                VALUES ('0', '0', '0', '0105000020D70800000100000001020000000F00000000A483D245B62E410028E0CCE40F0C4158D77B3712B62E41B17739726F0F0C413C07B088DCB52E41259C87170A0F0C41F3A05915A5B52E418A5A5E52B50E0C4175EC4D2F6CB52E4140F8D79F710E0C41BF47852A32B52E41EA9ADC633F0E0C41CA3A9F5CF7B42E4180D78EE81E0E0C41001B641CBCB42E41004ADE5D100E0C41B93940C180B42E418F2A45D9130E0C417236D1A245B42E41E4859E55290E0C41CF4E56180BB42E41450035B3500E0C4147673478D1B32E414F33F0B7890E0C41468B761799B32E41806AAA0FD40E0C41AF6B504962B32E4139D2AC4C2F0F0C410095A35E2DB32E41006251E89A0F0C41')
            
Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection closed 2020-03-02 15:12:27
Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection established 2020-03-02 15:12:27


...

Failure:

- Query run 2020-03-02 15:16:32.603000
	
                INSERT INTO working.sip_projs ("mft", "masteridfrom", "masteridto", "geom")
                VALUES ('9008919', '58000', '58059', '0105000020D7080000010000000102000000110000000062690CF6882F41000E6E6370B10F416F96D598FB882F411BD505153CB20F419485A03DFC882F41BA0918F708B30F41A6FC45F9F7882F41AA420127D5B30F410E37D3D5EE882F41BBC6C1C39EB40F41AF30CFE8E0882F41BAA06AF263B50F41D0EF0753CE882F4190697CE222B60F41103E4540B7882F41177D2DD2D9B60F418086E1E69B882F4100888D1287B70F417B6C48877C882F4186A7780B29B80F41C53E636B59882F415363643FBEB80F41D490E6E532882F41137DD74E45B90F417DDC905109882F41655FA9FBBCB90F417ABE5410DD872F416398EF2B24BA0F41B83A728AAE872F41E4F395EC79BA0F410128812D7E872F41FF189B73BDBA0F4180056F6B4C872F410066EC21EEBA0F41')
            
Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection closed 2020-03-02 15:16:32
Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection establis

12592it [04:20, 50.78it/s]

Failure:

- Query run 2020-03-02 15:16:39.316000
	
                INSERT INTO working.sip_projs ("mft", "masteridfrom", "masteridto", "geom")
                VALUES ('94218', '57324', '57327', '0105000020D70800000100000001020000001C000000009863BBBB702F41008AA8D623CD0D41009B2B54E4702F41002050EA26CB0D4100AE8C57E4702F41005C9EBB26CB0D4180C734DE18712F41006EE27132C90D41800158E818712F4100E83A1A32C90D41800461F218712F4100D0F2C931C90D4100F5B34859712F410066869E4CC70D418089195159712F4100C0595B4CC70D4180C33C5B59712F4100D4BA1E4CC70D41800ED90EA5712F4100A2CC7F7CC50D4180C9001EA5712F41001A25287CC50D4100F3CB2EA5712F410004DDD77BC50D41802BCA83FB712F4100E2201FC8C30D4100B0D792FB712F4100EE6FCEC7C30D41807E60A5FB712F410090B584C7C30D4180343F975B722F4100182B4335C20D41005E0AA85B722F4100DE07F934C20D4180631CBC5B722F410038DBB534C20D41804AA401C4722F4100B2C0D3C8C00D410009D51AC4722F41002A197CC8C00D4100DA6637C4722F410014D12BC8C00D410097A25633732F41002CB28486BF0D4100EA387833732F4100EC7C2686BF0D410050309D33732F4100423ECF8

12658it [04:22, 48.31it/s]



12615 rows added to working.sip_projs



### 4. Sipless Single Year MFT Speed Humps of Study Year

In [23]:
cdb.connect()

Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection established 2020-03-02 15:23:01


#### This query grabs all mfts that have dont have any sip projects and only have speed humps of the study year

In [24]:
study_group = cdb.query("""

--All no sip {my_study_year} Data

drop table if exists working.speed_hump_focus_group; 

create table working.speed_hump_focus_group as 

with data1 as(

--selecting all speed hump data from {my_study_year}
select distinct mft, masteridfrom, masteridto, date_installed, on_st,from_st, to_st, lion.geom, lion.segmentid 
from v_clean_speed_humps sh
join archive."18d.2019-11-13_lion" lion
on sh.segmentid = lion.segmentid 
where left(date_installed::text,4)::int = {my_study_year}
)

, data2 as(

select distinct coalesce(lion.mft,0) mft, coalesce(lion.masteridfrom,0) masteridfrom, coalesce(lion.masteridto,0) masteridto, lion.geom
from v_clean_speed_humps sh
join archive."18d.2019-11-13_lion" lion
on sh.segmentid = lion.segmentid 
where left(date_installed::text, 4)::int between {study_year_min} and {study_year_max}
and left(date_installed::text, 4)::int != {my_study_year}
)

--selecting all data that does not intersect with any other data of other years. 
select distinct data1.mft 
                ,data1.masteridfrom 
                ,data1.masteridto
                ,lion.segmentid::int 
                ,lion.nodeidfrom::int  
                ,lion.nodeidto::int
                ,on_st
                ,from_st 
                ,to_st
                ,lion.geom
from data1 
join archive."18d.2019-11-13_lion" lion
on data1.segmentid = lion.segmentid 
where data1.mft not in(select mft from data2)
        and data1.masteridfrom not in ((select masteridfrom mid from data2)
                      union 
                     (select masteridto mid from data2))
        and data1.masteridto not in   ((select masteridfrom mid from data2)
                      union 
                     (select masteridto mid from data2))
        and data1.mft not in (select mft::int from working.sip_projs)
        and data1.masteridfrom not in ((select masteridfrom::int from working.sip_projs)
                      union 
                      (select masteridto::int from working.sip_projs))
        and data1.masteridto not in   ((select masteridfrom::int from working.sip_projs)
                      union 
                      (select masteridto::int from working.sip_projs));

grant all on working.speed_hump_focus_group to public;

--select * from working.speed_hump_focus_group; 

    
    """.format(my_study_year=my_study_year,study_year_min=study_year_min,study_year_max=study_year_max) 
    )



Query run in 539000 microseconds


In [20]:
study_group

### Study Group Nodeids

In [16]:
cdb.connect()

Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection established 2020-03-02 15:18:39


In [17]:
study_group_nodes = cdb.dfquery("""

select distinct nodeidfrom "NODEID" from working.speed_hump_study_group

union 

select distinct nodeidto "NODEID" from working.speed_hump_study_group

"""
)

In [ ]:
#study_group_nodes.to_csv('study_group_nodes.csv')

### 5. Control Group: 
#### All mfts that do not intersect with any mfts that have speed humps installed within study year range

In [28]:
cdb.connect()

Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection established 2020-03-02 15:26:33


In [29]:
control_group = cdb.dfquery("""



--No sip control group

drop table if exists working.speed_hump_control_group; 

create table working.speed_hump_control_group as 

with data as (


select distinct coalesce(lion.mft,0) mft, coalesce(lion.masteridfrom,0) masteridfrom, coalesce(lion.masteridto,0) masteridto, lion.geom
from v_clean_speed_humps sh
join archive."18d.2019-11-13_lion" lion
on sh.segmentid = lion.segmentid 
where left(date_installed::text, 4)::int between {study_year_min} and {study_year_max}

)



select distinct mft
               ,masteridfrom
               ,masteridto
               ,segmentid::int
               ,nodeidfrom::int
               ,nodeidto::int
               ,lion.geom
from archive."18d.2019-11-13_lion" lion
where mft not in (select mft from data)
  and masteridfrom not in ((select masteridfrom mid from data)
                            union 
                           (select masteridto mid from data))
  and masteridto not in   ((select masteridfrom mid from data)
                            union 
                           (select masteridto mid from data))
  and mft not in (select mft::int from working.sip_projs)
  and masteridfrom not in ((select masteridfrom::int from working.sip_projs)
                            union 
                           (select masteridto::int from working.sip_projs))
  and masteridto not in ((select masteridfrom::int from working.sip_projs)
                            union 
                           (select masteridto::int from working.sip_projs));
               
grant all on working.speed_hump_control_group to public;               
--select * from working.speed_hump_control_group


""".format(study_year_min=study_year_min,study_year_max=study_year_max) 
)


### Control Group Nodes


In [30]:
cdb.connect()

Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection established 2020-03-02 15:37:38


In [31]:
control_group_nodes = cdb.dfquery("""


select distinct nodeidfrom "NODEID" from working.speed_hump_control_group

union 

select distinct nodeidto "NODEID" from working.speed_hump_control_group

"""
)

In [32]:
#control_group_nodes.to_csv('control_group_nodes.csv')

### 6. All Injuries by node from FORMS and TAMS Data

In [18]:
sqldb.connect()

	Missing SQL Server Native Client 10.0                 datetime2 will not be interpreted correctly

Database connection (MS) to forms on dot55sql01 - user: arcgis 
Connection established 2020-02-25 14:48:23


In [45]:
node_inj = sqldb.dfquery("""



SELECT NODEID
      ,CASE WHEN yr between {study_year_min} and {study_year_min} + 2 THEN 'before' 
            WHEN yr between {study_year_max} - 2 and {study_year_max} THEN 'after' 
            WHEN yr = {my_study_year} THEN 'study_year'
            ELSE 'exclude' END AS PERIOD
      ,coalesce(sum(CASE WHEN Mode = 'ped' THEN INJ END),0) PED
      ,coalesce(sum(CASE WHEN Mode = 'bicycle' THEN INJ END),0) BI
      ,coalesce(sum(CASE WHEN Mode = 'mvo' THEN INJ END),0) MVO
      ,sum(INJ) INJURIES
      ,yr
FROM (  
        SELECT nodeid
              ,yr
              ,CASE WHEN PED_NONPED = 'Pedestrian' THEN 'ped'
                 WHEN PED_NONPED = 'Bicyclist' THEN 'bicycle'
                 WHEN PED_NONPED = 'Occupant' THEN 'mvo' END as Mode
              ,sum(CASE WHEN INJ_KILLED = 'Injured' THEN 1 else 0 END) AS INJ
        FROM (SELECT coalesce(c.nodeid, lion_node_number) nodeid
                    ,year(c.accident_dt) yr
                    ,ped_nonped
                    ,inj_killed 
              FROM [FORMS].[dbo].wc_accident_f c
              LEFT JOIN FORMS.dbo.WC_ACCIDENT_VICTIM_F AS i
              ON c.INTEGRATION_ID=i.ACCIDENT_ID
              WHERE year(c.ACCIDENT_DT) > 2016 
                and coalesce(c.VOID_STATUS_CD , 'N') = 'N'
                and coalesce(nonmv, 0) = 0
                and coalesce(nodeid, lion_node_number) is not null 
              --unioning all crashes with null nodeid

              UNION

              SELECT s.nodeid
                    ,year(c.accident_dt) yr
                    ,ped_nonped
                    ,inj_killed  
              FROM [FORMS].[dbo].wc_accident_f c
              JOIN [FORMS].[dbo].[v_IntersectionStreetNames_Gen] s
              ON lower(ltrim(rtrim(c.SRC_ON_STREET))) = lower(ltrim(rtrim(s.street_1))) 
              and
              lower(ltrim(rtrim(c.SRC_cross_STREET))) = lower(ltrim(rtrim(s.street_2)))
              and 
              CASE WHEN C.[SRC_POLICE_PRECINCT] BETWEEN 1 AND 34 THEN 1
                   WHEN C.[SRC_POLICE_PRECINCT] BETWEEN 40 AND 52 THEN 2
                   WHEN C.[SRC_POLICE_PRECINCT] BETWEEN 60 AND 94 THEN 3
                   WHEN C.[SRC_POLICE_PRECINCT] BETWEEN 100 AND 115 THEN 4
                   WHEN C.[SRC_POLICE_PRECINCT] BETWEEN 120 AND 123 THEN 5 END 
              in (left(b7sc_2, 1), left(b7sc_1, 1))
              LEFT JOIN FORMS.dbo.WC_ACCIDENT_VICTIM_F AS i
              ON c.INTEGRATION_ID=i.ACCIDENT_ID
              WHERE year(c.ACCIDENT_DT) > 2016
                and coalesce(c.VOID_STATUS_CD , 'N') = 'N'
                and coalesce(nonmv, 0) = 0 
                and coalesce(c.nodeid, lion_node_number) is null
        ) all_forms
        GROUP BY nodeid, yr,
        CASE WHEN PED_NONPED = 'Pedestrian' THEN 'ped'
             WHEN PED_NONPED = 'Bicyclist' THEN 'bicycle'
             WHEN PED_NONPED = 'Occupant' THEN 'mvo' END

        UNION 

        SELECT crashes.nodeid
              ,crashes.yr
              ,CASE WHEN v.PED_NONPED = 'P' THEN 'ped'
                    WHEN v.PED_NONPED = 'B' THEN 'bicycle'
                    WHEN v.PED_NONPED = 'N' THEN 'mvo' 
                    ELSE 'STUDY YEAR' END AS Mode 
              ,coalesce(sum(v.INJURED_COUNT),0) INJ
        FROM(--selecting tams crashes
             SELECT DISTINCT c.nodeid
                            ,c.anum_pct
                            ,c.anum_seq
                            ,c.anum_yy
                            ,yr
             FROM (SELECT nodeid
                          ,anum_pct
                          ,anum_seq
                          ,anum_yy
                          ,year(occurrence_datetime) yr
                    FROM DataWarehouse.dbo.AIS_PD_Core_F  
                    WHERE (year(occurrence_datetime) between {study_year_min} and 2016)
                       --and year(occurrence_datetime) != {my_study_year}
                       and nodeid > 0
                    ) AS c
            JOIN DataWarehouse.dbo.AIS_PD_Locx_F AS l
            ON c.ANUM_PCT = l.ANUM_PCT and c.ANUM_SEQ = l.ANUM_SEQ and c.ANUM_YY = l.ANUM_YY

            UNION

            --unioning to account for tams crashes with nodeid = 0/street name combos with multiple potential nodes
            SELECT DISTINCT lu.nodeid, c.anum_pct, c.anum_seq, c.anum_yy, yr
            FROM (SELECT anum_pct, anum_seq, anum_yy, year(occurrence_datetime) yr 
                  FROM DataWarehouse.dbo.AIS_PD_Core_F 
                  WHERE (year(occurrence_datetime) between {study_year_min} and 2016)
                     --and year(occurrence_datetime) != {my_study_year}
                  ) AS c
            JOIN DataWarehouse.dbo.AIS_PD_Locx_F AS l
            ON c.ANUM_PCT = l.ANUM_PCT and c.ANUM_SEQ = l.ANUM_SEQ and c.ANUM_YY = l.ANUM_YY
            JOIN DataWarehouse.dbo.nypd_location_lookup_F AS lu
            ON c.ANUM_PCT = lu.ANUM_PCT and l.ON_STREET in (lu.[ON_STREET], lu.CROSS_STREET) and l.CROSS_STREET in (lu.[ON_STREET], lu.CROSS_STREET)
            ---AG changed above from: on c.ANUM_PCT = lu.ANUM_PCT and l.ON_STREET = lu.[ON_STREET] and l.CROSS_STREET = lu.CROSS_STREET
        ) crashes
        LEFT JOIN DataWarehouse.dbo.AIS_PD_Victim_F AS v
        on crashes.ANUM_PCT = v.ANUM_PCT and crashes.ANUM_SEQ = v.ANUM_SEQ and crashes.ANUM_YY = v.ANUM_YY
        GROUP BY crashes.nodeid
                ,crashes.yr
                ,CASE WHEN v.PED_NONPED = 'P' THEN 'ped'
                      WHEN v.PED_NONPED = 'B' THEN 'bicycle'
                      WHEN v.PED_NONPED = 'N' THEN 'mvo' 
                      ELSE 'STUDY YEAR' END 

        ) data 

GROUP BY NODEID
        ,CASE WHEN yr between {study_year_min} and {study_year_min} + 2 THEN 'before' 
            WHEN  yr between {study_year_max} - 2 and {study_year_max} THEN 'after' ELSE 'exclude' END
        ,yr
""".format(my_study_year=my_study_year,study_year_min=study_year_min,study_year_max=study_year_max))


	Missing SQL Server Native Client 10.0                 datetime2 will not be interpreted correctly



In [50]:
print timestamp

2020-03-02 16:42


In [118]:
sg = node_inj.loc[node_inj['PERIOD']!='exclude']

In [121]:
study_group_inj = study_group_nodes.merge(sg, on='NODEID', how='left')

In [123]:
s = study_group_inj.groupby(['yr'])['INJURIES'].sum()

In [125]:
cg = node_inj.loc[node_inj['PERIOD']!='exclude']

In [126]:
control_group_inj = control_group_nodes.merge(cg, on='NODEID', how='left')

In [127]:
c = control_group_inj.groupby(['yr'])['INJURIES'].sum()

In [128]:
c

yr
2011.0    31792.0
2012.0    31693.0
2013.0    32202.0
2014.0    29448.0
2015.0    30304.0
2016.0    33906.0
2017.0    14924.0
Name: INJURIES, dtype: float64

In [132]:
c=pd.DataFrame(c)

In [133]:
s= pd.DataFrame(s)

In [134]:
tot= pd.concat([c,s], axis=0).sort_values('yr')

In [139]:
total = pd.DataFrame(tot.groupby(['yr'])['INJURIES'].sum())

In [145]:
writer = pd.ExcelWriter('speed_hump_analysis_302020.xlsx',engine = 'xlsxwriter') #opening new excel file to write data to

In [147]:
s.to_excel(writer, sheet_name='study_group')
c.to_excel(writer, sheet_name='control_group')
total.to_excel(writer, sheet_name='total')
writer.save() 

In [135]:
total.to_csv('speed_hump_analysis_302020.csv')

IOError: [Errno 13] Permission denied: 'speed_hump_analysis_302020.csv'

In [101]:
tot


,INJURIES
yr,
2011.0,31792.0
2012.0,31693.0
2013.0,32202.0
2014.0,400.0
2015.0,30304.0
2016.0,33906.0
2017.0,14924.0


### 7. Before-Afters

### All Study Group Injuries before and after Study Year

In [ ]:
study_group_inj = study_group_nodes.merge(node_inj, on='NODEID', how='left')

#### Before

In [70]:
s_inj_b = study_group_inj.loc[study_group_inj['PERIOD'] == 'before'].INJURIES.sum()

In [71]:
print "{} injuries before {}".format(s_inj_b,my_study_year)

0.0 injuries before 2014


#### After

In [33]:
s_inj_a = study_group_inj.loc[study_group_inj['PERIOD'] == 'after'].INJURIES.sum()

In [34]:
print "{} injuries before {}".format(s_inj_a,my_study_year)

978.0 injuries before 2014


In [43]:
study = [['PED', study_group_inj.loc[study_group_inj['PERIOD'] == 'before'].PED.sum(), study_group_inj.loc[study_group_inj['PERIOD'] == 'after'].PED.sum()], 
         ['BI', study_group_inj.loc[study_group_inj['PERIOD'] == 'before'].BI.sum(), study_group_inj.loc[study_group_inj['PERIOD'] == 'after'].BI.sum()],
         ['MVO', study_group_inj.loc[study_group_inj['PERIOD'] == 'before'].MVO.sum(), study_group_inj.loc[study_group_inj['PERIOD'] == 'after'].MVO.sum()]] 
  
# Create the pandas DataFrame 
study_df = pd.DataFrame(study,columns = ['Mode', '2010-2013', '2015-2017'])
study_df['% Change']  = ((study_df['2015-2017'] - study_df['2010-2013'])/study_df['2010-2013'])*100
  
# print dataframe. 
study_df

,Mode,2010-2013,2015-2017,% Change
0,PED,340.0,267.0,-21.470588
1,BI,170.0,134.0,-21.176471
2,MVO,714.0,577.0,-19.187675


### All Control Group Injuries before and after Study Year

In [36]:
control_group_inj = control_group_nodes.merge(node_inj, on='NODEID', how='left')

#### Before

In [37]:
c_inj_b = control_group_inj.loc[control_group_inj['PERIOD'] == 'before'].INJURIES.sum()

In [38]:
print "{} injuries before {}".format(c_inj_b,my_study_year)

95819.0 injuries before 2014


#### After

In [39]:
c_inj_a = control_group_inj.loc[control_group_inj['PERIOD'] == 'after'].INJURIES.sum()

In [40]:
print "{} injuries before {}".format(c_inj_a,my_study_year)

79235.0 injuries before 2014


In [42]:
control = [['PED', control_group_inj.loc[control_group_inj['PERIOD'] == 'before'].PED.sum(), control_group_inj.loc[control_group_inj['PERIOD'] == 'after'].PED.sum()], 
           ['BI', control_group_inj.loc[control_group_inj['PERIOD'] == 'before'].BI.sum(), control_group_inj.loc[control_group_inj['PERIOD'] == 'after'].BI.sum()],
           ['MVO', control_group_inj.loc[control_group_inj['PERIOD'] == 'before'].MVO.sum(), control_group_inj.loc[control_group_inj['PERIOD'] == 'after'].MVO.sum()]] 
  
# Create the pandas DataFrame 
control_df = pd.DataFrame(control, columns = ['Mode', '2010-2013', '2015-2017']) 
control_df['% Change']  = ((control_df['2015-2017'] - control_df['2010-2013'])/control_df['2010-2013'])*100
# print dataframe. 
control_df

,Mode,2010-2013,2015-2017,% Change
0,PED,21810.0,17609.0,-19.261807
1,BI,7224.0,7497.0,3.779070
2,MVO,66785.0,54129.0,-18.950363


In [ ]:
writer = pd.ExcelWriter('speed_hump_analysis.xlsx',engine = 'xlsxwriter') #opening new excel file to write data to

In [ ]:
study_df.to_excel(writer, sheet_name='Sheet1', index = False)
control_df.to_excel(writer, sheet_name='Sheet1', startrow=5, index = False)
writer.save() 

In [ ]:
cdb.connect()

In [ ]:
cdb.query("""

drop table if exists working.sip_projs; 

drop table if exists working.speed_hump_focus_group; 

drop table if exists working.speed_hump_control_group; 

drop table if exists working.speed_hump_control_group; 

"""
)